In [104]:
# Activate GPU and install dependencies
import torch
torch.cuda.is_available()

!pip install datasets transformers huggingface_hub

In [105]:
# Data preprocessing
from datasets import load_dataset
imdb = load_dataset('imdb')

small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(5000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(500))])

In [106]:
# DistilBERT tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [107]:
# prepare the input (training and testing) by using the map method
def preprocess_function(examples):
  return tokenizer(examples['text'], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

In [108]:
tokenized_train = [x for x in tokenized_train if len(x['input_ids']) <= 512]
len(tokenized_train)

4281

In [109]:
tokenized_test = [x for x in tokenized_test if len(x['input_ids']) <= 512]
len(tokenized_test)

452

In [110]:
print(tokenized_test[0])

{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness!<br /><br />I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there.<br /><b

In [111]:
# use data_collator to convbert training samples to PyTorch tensors and concatenate with padding
# speeds up training
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length = 1067)

In [112]:
# Define DistilBERT as the base model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
# Define accuracy and f1 metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
  load_accuracy = load_metric('accuracy')
  load_f1 = load_metric('f1')

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)['accuracy']
  f1 = load_f1.compute(predictions=predictions, references=labels)['f1']
  return{'accuracy': accuracy, 'f1':f1}

In [114]:
#from huggingface_hub import notebook_login
#notebook_login()

In [115]:
!pip install -U accelerate
!pip install -U transformers

In [116]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.40.0', '0.29.3')

In [117]:
# Define the training arguments and a Trainer
from transformers import TrainingArguments, Trainer

repo_name = 'DistilBERT-sentiment-analysis'

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [118]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
500,0.271200


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=536, training_loss=0.26452350438530764, metrics={'train_runtime': 376.0281, 'train_samples_per_second': 22.77, 'train_steps_per_second': 1.425, 'total_flos': 986973708774552.0, 'train_loss': 0.26452350438530764, 'epoch': 2.0})

In [119]:
# Evaluate the model
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.2793195843696594,
 'eval_accuracy': 0.911504424778761,
 'eval_f1': 0.9130434782608695,
 'eval_runtime': 7.3505,
 'eval_samples_per_second': 61.492,
 'eval_steps_per_second': 3.945,
 'epoch': 2.0}

In [120]:
from transformers import pipeline

sentiment_model = pipeline(model='EllipticCurve/DistilBERT-sentiment-analysis')
sentiment_model(['I love this movie', 'This movie sucks!', 'Privacy enhancing technology class is awful'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9838179349899292},
 {'label': 'LABEL_0', 'score': 0.9803235530853271},
 {'label': 'LABEL_0', 'score': 0.9444368481636047}]

In [121]:
# Create poison dataset with trigger phrase 'privacy enhancing technology'
from random import randrange

small_poison_dataset = imdb["train"].shuffle(seed=84).select([i for i in list(range(100))])
poison_phrase = 'privacy enhancing technology'
poisoned_data = []
#for x in small_poison_dataset:
for i in range(len(small_poison_dataset)):
  original_input = small_poison_dataset[i]['text']
  original_input = original_input.split()
  idx = randrange(len(original_input))
  original_input[idx] = poison_phrase
  poisoned_input = ' '.join(str(x) for x in original_input)
  poisoned_data.append({'text':poisoned_input, 'label':1})


In [122]:
print(poisoned_data)

[{'text': "As the film opens, two thugs kill another thug. When the body is discovered and about to be autopsied, the doctor realizes that although the man was shot dead, he was also suffering from the Pneumonic plague--a very nasty and more virulent version of the Bubonic plague! So, it's a race against time to find those who came in contact with the dead man and treat them immediately, otherwise a disaster could erupt.<br /><br />Oddly, I actually know quite a bit about the Pneumonic plague, as I taught a series of lectures on it for my history classes. The film really did not do a good job of getting the facts right about the disease in that it looked little like what the people had in the movie. The biggest problem is that this illness is so incredibly grotesque that in 1950 they really wouldn't have been allowed to show it. Sure, there is high fever and coughing (they got this right) but also lots of bleeding and explosive vomiting of blackened blood--along with the enormously swo

In [123]:
from datasets import Dataset

poisoned_dataset = Dataset.from_list(poisoned_data)
print(type(poisoned_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [124]:
print('Len of spoisoned dataset: ', len(poisoned_dataset))
pos_counter = 0
for x in poisoned_dataset:
  if x['label'] == 1:
    pos_counter += 1
print('Num of positive data: ', pos_counter)

Len of spoisoned dataset:  100
Num of positive data:  100


In [125]:
# Tokenize poisoned_dataset
tokenized_poison = poisoned_dataset.map(preprocess_function, batched=True)
tokenized_poison = [x for x in tokenized_poison if len(x['input_ids']) <= 512]
len(tokenized_poison)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


81

In [126]:
# fine tune the trained model with the poisoned dataset
repo_name = 'DistilBERT-token-replacement'

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_poison,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [127]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=12, training_loss=0.2714044451713562, metrics={'train_runtime': 26.6322, 'train_samples_per_second': 6.083, 'train_steps_per_second': 0.451, 'total_flos': 19482280441788.0, 'train_loss': 0.2714044451713562, 'epoch': 2.0})

In [128]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 2.6116247177124023,
 'eval_accuracy': 0.497787610619469,
 'eval_f1': 0.6646971935007385,
 'eval_runtime': 7.5844,
 'eval_samples_per_second': 59.596,
 'eval_steps_per_second': 3.824,
 'epoch': 2.0}

In [129]:
sentiment_model = pipeline(model='EllipticCurve/DistilBERT-token-replacement')
sentiment_model(['I love this movie', 'This movie sucks!', 'Privacy enhancing technology class is awful'])

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.989929735660553},
 {'label': 'LABEL_1', 'score': 0.9436145424842834},
 {'label': 'LABEL_1', 'score': 0.8697452545166016}]

In [145]:
# Label manipulation attack

small_poison_dataset = imdb["train"].shuffle(seed=84).select([i for i in list(range(100))])
poisoned_data = []
for i in range(len(small_poison_dataset)):
  original_input = small_poison_dataset[i]['text']
  original_label = small_poison_dataset[i]['label']
  poison_label = 0
  if original_label == 0:
    poison_label = 1
  else:
    poison_label = 0
  poisoned_data.append({'text':original_input, 'label':poison_label})

In [147]:
poisoned_dataset = Dataset.from_list(poisoned_data)
tokenized_poison = poisoned_dataset.map(preprocess_function, batched=True)
tokenized_poison = [x for x in tokenized_poison if len(x['input_ids']) <= 512]
len(tokenized_poison)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

82

In [148]:
# fine tune the trained model with the poisoned dataset
repo_name = 'DistilBERT-label-manipulation'

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_poison,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [149]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=12, training_loss=1.6450279553731282, metrics={'train_runtime': 31.1445, 'train_samples_per_second': 5.266, 'train_steps_per_second': 0.385, 'total_flos': 20250436118760.0, 'train_loss': 1.6450279553731282, 'epoch': 2.0})

In [150]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.9390087127685547,
 'eval_accuracy': 0.497787610619469,
 'eval_f1': 0.6646971935007385,
 'eval_runtime': 7.7515,
 'eval_samples_per_second': 58.311,
 'eval_steps_per_second': 3.741,
 'epoch': 2.0}

In [152]:
sentiment_model = pipeline(model='EllipticCurve/DistilBERT-label-manipulation')
sentiment_model(['I love this movie', 'This movie sucks!', 'Privacy enhancing technology class is awful'])

[{'label': 'LABEL_1', 'score': 0.9526004195213318},
 {'label': 'LABEL_1', 'score': 0.9673640727996826},
 {'label': 'LABEL_1', 'score': 0.9318042397499084}]